In [45]:
import pandas as pd
import numpy as np

## 预定义路径

In [221]:
filename_traffic_nov = 'C:/program2/tensorflow/data/traffic_nov.csv'# 7:00
filename_traffic_dec = 'C:/program2/tensorflow/data/traffic_dec.csv'# 7:00
filename_set_nov = 'C:/program2/tensorflow/data/set_consumption_nov.csv' # 0:00 
filename_set_dec = 'C:/program2/tensorflow/data/set_consumption_dec.csv' # 0:00 
filename_tem_nov = 'C:/program2/tensorflow/data/temperature_nov_resample.csv'# 0:00  36个id
filename_tem_dec = 'C:/program2/tensorflow/data/temperature_dec_resample.csv'# 0:00  36个id
filename_sms_call_dec = 'C:/program2/tensorflow/data/sms_call_dec.csv'# 7:00
filename_sms_call_nov = 'C:/program2/tensorflow/data/sms_call_nov.csv'# 7:00

filename_traffic_de_noising_nov = 'C:/program2/tensorflow/data/traffic_de_noising_nov.csv'# 7:00
filename_traffic_correlation_nov = 'C:/program2/tensorflow/data/traffic_correlation_nov.csv'# 7:00



## 读取流量数据，电力数据

In [3]:
traffic_nov = pd.read_csv(filename_traffic_nov)#'id_9991', 'id_9992', 'id_9993', 'id_9994', 'id_9995', 'id_9996'
traffic_nov = traffic_nov.drop('Unnamed: 0',1)
traffic_nov = traffic_nov.drop('Unnamed: 0.1',1)
traffic_nov = traffic_nov.drop('time',1)
traffic_index = traffic_nov.columns

set_nov = pd.read_csv(filename_set_nov)#'10867', '10868', '10980', '10981', '10982', '10983', '10984', '11097'
set_nov = set_nov.drop('Unnamed: 0',1)
set_nov = set_nov.drop('Unnamed: 0.1',1)
set_nov = set_nov.drop('Unnamed: 0.1.1',1)
set_index = set_nov.columns

## 去噪函数

In [194]:
def de_noising(traffic_data,set_data):
    fft = np.fft.fft(traffic_data)[0:4320]  #调用fft变换算法计算频域波形
    stop_freq = np.linspace(50,250,201)     #设置高通滤波器的截至频率
    ori_corr = correlation(traffic_data,set_data) #将去噪前的相关系数做初始的最大相关系数
    max_corr = ori_corr
    optmizer_stop_freq = 0
    optmizer_de_noising = 0
    #print(ori_corr)
    flag = 0
    for i in range(len(stop_freq)):         #开始选择最优截至频率  目标是相关系数最大
        #print(i)
        local_fft = fft.copy()
        local_fft[int(stop_freq[i]):] = 0
        ifft = np.fft.ifft(local_fft)
        corr = correlation(abs(pd.Series(ifft)),set_data)
        #print(corr)
        if abs(corr) >= abs(max_corr):
            max_corr = corr
            optmizer_stop_freq = stop_freq[i]
            optmizer_de_noising = abs(ifft)
            flag = 1
        if flag == 0:
            optmizer_de_noising = traffic_data.values
    return (ori_corr,max_corr,optmizer_stop_freq,optmizer_de_noising)   # 返回原相关系数，最大相关系数，最佳截至频率，最佳去噪序列

## 相关系数函数

In [195]:
def correlation(x,y):
    return x.corr(y, method = 'pearson')

## 正式对流量数据进行去噪 保存成新的DataFrame

In [196]:
# -*- coding: UTF-8 -*-
import numpy as np

traffic_de_noising = pd.DataFrame(index = [i for i in range(0,4320)])
traffic_correlation = pd.DataFrame(index = [i for i in range(0,3)])

for j in range(len(set_index)):
    if 'id_'+str(set_index[j]) not in traffic_index:
        continue
    y1 = traffic_nov['id_'+str(set_index[j])]
    y2 = set_nov[str(set_index[j])]
    ori_corr,max_corr,optmizer_stop_freq,optmizer_de_noising = de_noising(y1,y2)
    traffic_de_noising_perid = pd.DataFrame(optmizer_de_noising, columns = [str(set_index[j])])  #某个地区的去噪流量值
    traffic_correlation_perid = pd.DataFrame([ori_corr,max_corr,optmizer_stop_freq], columns = [str(set_index[j])])  #某个地区的去噪前后相关系数
    traffic_de_noising = pd.concat((traffic_de_noising,traffic_de_noising_perid),axis = 1)
    traffic_correlation = pd.concat((traffic_correlation,traffic_correlation_perid),axis = 1)
    print('the '+str(j)+'th finished ')

the 0th finished 
the 1th finished 
the 2th finished 
the 3th finished 
the 4th finished 
the 5th finished 
the 6th finished 
the 7th finished 
the 8th finished 
the 9th finished 
the 10th finished 
the 11th finished 
the 12th finished 
the 13th finished 
the 15th finished 
the 16th finished 
the 17th finished 
the 18th finished 
the 19th finished 
the 20th finished 
the 22th finished 
the 23th finished 
the 24th finished 
the 25th finished 
the 26th finished 
the 27th finished 
the 28th finished 
the 29th finished 
the 30th finished 
the 32th finished 
the 33th finished 
the 34th finished 
the 35th finished 
the 36th finished 
the 37th finished 
the 38th finished 
the 39th finished 
the 40th finished 
the 41th finished 
the 42th finished 
the 43th finished 
the 44th finished 
the 45th finished 
the 46th finished 
the 47th finished 
the 48th finished 
the 49th finished 
the 50th finished 
the 51th finished 
the 52th finished 
the 53th finished 
the 54th finished 
the 55th finished 
the

the 418th finished 
the 419th finished 
the 420th finished 
the 421th finished 
the 422th finished 
the 423th finished 
the 424th finished 
the 425th finished 
the 426th finished 
the 427th finished 
the 428th finished 
the 429th finished 
the 430th finished 
the 431th finished 
the 432th finished 
the 433th finished 
the 434th finished 
the 435th finished 
the 436th finished 
the 437th finished 
the 438th finished 
the 439th finished 
the 440th finished 
the 441th finished 
the 442th finished 
the 443th finished 
the 444th finished 
the 445th finished 
the 446th finished 
the 447th finished 
the 448th finished 
the 449th finished 
the 450th finished 
the 451th finished 
the 452th finished 
the 453th finished 
the 454th finished 
the 455th finished 
the 456th finished 
the 457th finished 
the 458th finished 
the 459th finished 
the 460th finished 
the 461th finished 
the 462th finished 
the 463th finished 
the 464th finished 
the 465th finished 
the 466th finished 
the 467th finished 


the 848th finished 
the 849th finished 
the 850th finished 
the 851th finished 
the 852th finished 
the 853th finished 
the 854th finished 
the 855th finished 
the 856th finished 
the 857th finished 
the 858th finished 
the 859th finished 
the 860th finished 
the 861th finished 
the 862th finished 
the 863th finished 
the 864th finished 
the 865th finished 
the 866th finished 
the 867th finished 
the 868th finished 
the 869th finished 
the 870th finished 
the 871th finished 
the 872th finished 
the 873th finished 
the 874th finished 
the 875th finished 
the 876th finished 
the 877th finished 
the 878th finished 
the 879th finished 
the 880th finished 
the 881th finished 
the 882th finished 
the 883th finished 
the 884th finished 
the 885th finished 
the 886th finished 
the 887th finished 
the 888th finished 
the 889th finished 
the 890th finished 
the 891th finished 
the 892th finished 
the 893th finished 
the 894th finished 
the 895th finished 
the 896th finished 
the 897th finished 


the 1250th finished 
the 1252th finished 
the 1253th finished 
the 1254th finished 
the 1255th finished 
the 1256th finished 
the 1257th finished 
the 1258th finished 
the 1259th finished 
the 1260th finished 
the 1261th finished 
the 1262th finished 
the 1263th finished 
the 1264th finished 
the 1265th finished 
the 1266th finished 
the 1267th finished 
the 1268th finished 
the 1269th finished 
the 1270th finished 
the 1271th finished 
the 1272th finished 
the 1273th finished 
the 1274th finished 
the 1275th finished 
the 1276th finished 
the 1277th finished 
the 1278th finished 
the 1279th finished 
the 1281th finished 
the 1282th finished 
the 1284th finished 
the 1285th finished 
the 1286th finished 
the 1287th finished 
the 1288th finished 
the 1289th finished 
the 1290th finished 
the 1291th finished 
the 1292th finished 
the 1293th finished 
the 1294th finished 
the 1295th finished 
the 1296th finished 
the 1297th finished 
the 1298th finished 
the 1299th finished 
the 1300th fi

the 1654th finished 
the 1655th finished 
the 1656th finished 
the 1657th finished 
the 1658th finished 
the 1659th finished 
the 1660th finished 
the 1661th finished 
the 1662th finished 
the 1663th finished 
the 1664th finished 
the 1665th finished 
the 1666th finished 
the 1667th finished 
the 1668th finished 
the 1669th finished 
the 1670th finished 
the 1671th finished 
the 1672th finished 
the 1673th finished 
the 1674th finished 
the 1675th finished 
the 1676th finished 
the 1677th finished 
the 1678th finished 
the 1679th finished 
the 1680th finished 
the 1681th finished 
the 1682th finished 
the 1683th finished 
the 1684th finished 
the 1685th finished 
the 1686th finished 
the 1687th finished 
the 1688th finished 
the 1689th finished 
the 1690th finished 
the 1691th finished 
the 1692th finished 
the 1693th finished 
the 1694th finished 
the 1695th finished 
the 1696th finished 
the 1697th finished 
the 1698th finished 
the 1699th finished 
the 1700th finished 
the 1701th fi

In [203]:
traffic_de_noising.to_csv(filename_traffic_de_noising_nov)

In [214]:
traffic_correlation.to_csv(filename_traffic_correlation_nov)

## 计算相关系数增加的量与增加率

In [217]:
def coor_add(traffic_corr):
    a = traffic_corr.T
    a[0] = abs(a[0])
    a[1] = abs(a[1])
    add = (a[1]-a[0])
    add = add.sum()
    add_rate = add/1978
    return add,add_rate

In [219]:
add,add_rate = coor_add(traffic_correlation)
print('相关性平均提高了：',add_rate)

相关性平均提高了： 0.06604745979025378


## 读取电话短信数据

In [222]:
sms_call_nov = pd.read_csv(filename_sms_call_nov)
sms_call_nov = sms_call_nov.drop('Unnamed: 0',1)
sms_call_nov = sms_call_nov.drop('Unnamed: 0.1',1)
call_index = sms_call_nov.columns[:6259]
sms_index = sms_call_nov.columns[6259:-1]

## 对电话数据进行去噪

In [223]:
# -*- coding: UTF-8 -*-
import numpy as np

call_de_noising = pd.DataFrame(index = [i for i in range(0,4320)])
call_correlation = pd.DataFrame(index = [i for i in range(0,3)])

for j in range(len(set_index)):
    if 'call'+str(set_index[j]) not in call_index:
        continue
    y1 = sms_call_nov['call'+str(set_index[j])]
    y2 = set_nov[str(set_index[j])]
    ori_corr,max_corr,optmizer_stop_freq,optmizer_de_noising = de_noising(y1,y2)
    call_de_noising_perid = pd.DataFrame(optmizer_de_noising, columns = [str(set_index[j])])  #某个地区的去噪电话值
    call_correlation_perid = pd.DataFrame([ori_corr,max_corr,optmizer_stop_freq], columns = [str(set_index[j])])  #某个地区的去噪前后相关系数
    call_de_noising = pd.concat((call_de_noising,call_de_noising_perid),axis = 1)
    call_correlation = pd.concat((call_correlation,call_correlation_perid),axis = 1)
    if j == 1:
        break
    print('the '+str(j)+'th finished ')

the 0th finished 
the 1th finished 
the 2th finished 
the 3th finished 
the 4th finished 
the 5th finished 
the 6th finished 
the 7th finished 
the 8th finished 
the 9th finished 
the 10th finished 
the 11th finished 
the 12th finished 
the 13th finished 
the 15th finished 
the 16th finished 
the 17th finished 
the 18th finished 
the 19th finished 
the 20th finished 
the 22th finished 
the 23th finished 
the 24th finished 
the 25th finished 
the 26th finished 
the 27th finished 
the 28th finished 
the 29th finished 
the 30th finished 
the 32th finished 
the 33th finished 
the 34th finished 
the 35th finished 
the 36th finished 
the 37th finished 
the 38th finished 
the 39th finished 
the 40th finished 
the 41th finished 
the 42th finished 
the 43th finished 
the 44th finished 
the 45th finished 
the 46th finished 
the 47th finished 
the 48th finished 
the 49th finished 
the 50th finished 
the 51th finished 
the 52th finished 
the 53th finished 
the 54th finished 
the 55th finished 
the

the 419th finished 
the 420th finished 
the 421th finished 
the 422th finished 
the 423th finished 
the 424th finished 
the 425th finished 
the 426th finished 
the 427th finished 
the 428th finished 
the 429th finished 
the 430th finished 
the 431th finished 
the 432th finished 
the 433th finished 
the 434th finished 
the 435th finished 
the 436th finished 
the 437th finished 
the 438th finished 
the 439th finished 
the 440th finished 
the 441th finished 
the 442th finished 
the 443th finished 
the 444th finished 
the 445th finished 
the 446th finished 
the 447th finished 
the 448th finished 
the 449th finished 
the 450th finished 
the 451th finished 
the 452th finished 
the 453th finished 
the 454th finished 
the 455th finished 
the 456th finished 
the 457th finished 
the 458th finished 
the 459th finished 
the 460th finished 
the 461th finished 
the 462th finished 
the 463th finished 
the 464th finished 
the 465th finished 
the 466th finished 
the 467th finished 
the 468th finished 


the 850th finished 
the 851th finished 
the 852th finished 
the 853th finished 
the 854th finished 
the 855th finished 
the 856th finished 
the 857th finished 
the 858th finished 
the 859th finished 
the 860th finished 
the 861th finished 
the 862th finished 
the 863th finished 
the 864th finished 
the 865th finished 
the 866th finished 
the 867th finished 
the 868th finished 
the 869th finished 
the 870th finished 
the 871th finished 
the 872th finished 
the 873th finished 
the 874th finished 
the 875th finished 
the 876th finished 
the 877th finished 
the 878th finished 
the 879th finished 
the 880th finished 
the 881th finished 
the 882th finished 
the 883th finished 
the 884th finished 
the 885th finished 
the 886th finished 
the 887th finished 
the 888th finished 
the 889th finished 
the 890th finished 
the 891th finished 
the 892th finished 
the 893th finished 
the 894th finished 
the 895th finished 
the 896th finished 
the 897th finished 
the 898th finished 
the 899th finished 


the 1253th finished 
the 1254th finished 
the 1255th finished 
the 1256th finished 
the 1257th finished 
the 1258th finished 
the 1259th finished 
the 1260th finished 
the 1261th finished 
the 1262th finished 
the 1263th finished 
the 1264th finished 
the 1265th finished 
the 1266th finished 
the 1267th finished 
the 1268th finished 
the 1269th finished 
the 1270th finished 
the 1271th finished 
the 1272th finished 
the 1273th finished 
the 1274th finished 
the 1275th finished 
the 1276th finished 
the 1277th finished 
the 1278th finished 
the 1279th finished 
the 1281th finished 
the 1282th finished 
the 1284th finished 
the 1285th finished 
the 1286th finished 
the 1287th finished 
the 1288th finished 
the 1289th finished 
the 1290th finished 
the 1291th finished 
the 1292th finished 
the 1293th finished 
the 1294th finished 
the 1295th finished 
the 1296th finished 
the 1297th finished 
the 1298th finished 
the 1299th finished 
the 1300th finished 
the 1301th finished 
the 1302th fi

the 1656th finished 
the 1657th finished 
the 1658th finished 
the 1659th finished 
the 1660th finished 
the 1661th finished 
the 1662th finished 
the 1663th finished 
the 1664th finished 
the 1665th finished 
the 1666th finished 
the 1667th finished 
the 1668th finished 
the 1669th finished 
the 1670th finished 
the 1671th finished 
the 1672th finished 
the 1673th finished 
the 1674th finished 
the 1675th finished 
the 1676th finished 
the 1677th finished 
the 1678th finished 
the 1679th finished 
the 1680th finished 
the 1681th finished 
the 1682th finished 
the 1683th finished 
the 1684th finished 
the 1685th finished 
the 1686th finished 
the 1687th finished 
the 1688th finished 
the 1689th finished 
the 1690th finished 
the 1691th finished 
the 1692th finished 
the 1693th finished 
the 1694th finished 
the 1695th finished 
the 1696th finished 
the 1697th finished 
the 1698th finished 
the 1699th finished 
the 1700th finished 
the 1701th finished 
the 1702th finished 
the 1703th fi

In [224]:
call_de_noising

,155,272,273,276,277,389,390,393,506,507,...,10867,10868,10980,10981,10982,10983,10984,11097,11098,11099
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.410668,0.101707,NaN,NaN,0.160206,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,0.256465,0.096805,NaN,NaN,0.267746,0.024793,NaN,...,0.015324,0.211209,NaN,NaN,NaN,NaN,NaN,0.009260,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.045740,0.139262,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.343491,0.422928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,0.225530,NaN,NaN,NaN,NaN,NaN,...,0.062494,0.228516,0.211036,0.029989,NaN,NaN,NaN,0.026447,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.013685
7,NaN,NaN,NaN,NaN,0.134558,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,0.134558,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,0.207871,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [225]:
call_correlation

,155,272,273,276,277,389,390,393,506,507,...,10867,10868,10980,10981,10982,10983,10984,11097,11098,11099
0,0.332245,0.345411,0.343692,0.618161,0.721791,0.339052,0.362503,0.688582,0.356249,0.368285,...,0.024052,0.0301,0.045493,0.097022,0.008295,0.009854,-0.011896,0.054299,0.078494,0.085074
1,0.332245,0.345411,0.343692,0.618161,0.721791,0.339052,0.362503,0.688582,0.356249,0.368285,...,0.024052,0.0301,0.045493,0.097022,0.008295,0.009854,-0.011896,0.054299,0.078494,0.085074
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
